# Document for evaluating on full images using trained resnet

In [1]:
import albumentations as A
import numpy as np
import cv2
import torch
from fastai.vision.all import *
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

## Init setup

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DATASET_PATH = Path("/home/mikkel/Documents/experts_in_teams_proj/vision/data/fence_data/texel_data/categories")

### Replace the dataloader with full images each cropped to texel/no-texel size by making a custom class

In [3]:
class ImageAlbumentationsTransform(Transform):
    """Class for applying the transformations on the images
    """
    def __init__(self, aug): 
        self.aug = aug
           
    def encodes(self, img: PILImage):
        aug_img = self.aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

trans = A.Compose([
    A.HorizontalFlip(p=0.25),
    A.VerticalFlip(p=0.25),
    A.RandomBrightnessContrast(p=1),    
    A.RandomGamma(p=1),    
    A.CLAHE(p=1),
    
])

tfm = ImageAlbumentationsTransform(trans)

In [4]:
def label_func(fname):
    return 'texel' if 'true' in str(fname) else "no texel"

dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    get_y=label_func,
    item_tfms=[ToTensor, tfm],
    splitter=RandomSplitter(valid_pct=0.2, seed=100),
    batch_tfms = [IntToFloatTensor, Normalize.from_stats(*imagenet_stats)],
)

dls = dblock.dataloaders(DATASET_PATH, bs=12, num_workers=8)

### Loading trained parameters

In [5]:
learn = cnn_learner(dls, resnet50, metrics=accuracy, model_dir="/home/mikkel/Documents/experts_in_teams_proj/vision/texel_resnet50/models")
learn.load("/home/mikkel/Documents/experts_in_teams_proj/vision/texel_resnet50/models/model")

In [6]:
def sliding_window(image, stepSize, windowSize):
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [10]:
patches = list()

winW = 32
winH = 32
ori_img = cv2.imread('/home/mikkel/Documents/experts_in_teams_proj/vision/data/lorenz-fence-inspection-examples/images/Eskild_fig_3_16.jpg')
resized = cv2.resize(ori_img, (1280, 1280))
for (x, y, window) in sliding_window(resized, stepSize=32, windowSize=(winW, winH)):
    # if the window does not meet our desired window size, ignore it
    if window.shape[0] != winH or window.shape[1] != winW:
        continue
    # THIS IS WHERE YOU WOULD PROCESS YOUR WINDOW, SUCH AS APPLYING A
    # MACHINE LEARNING CLASSIFIER TO CLASSIFY THE CONTENTS OF THE
    # WINDOW
    # since we do not have a classifier, we'll just draw the window
    patch = resized[y:y+winH, x:x+winW]
    patches.append(PILImage.create(patch))

In [11]:
trans=transforms.Compose([transforms.Resize(128),
                          transforms.ToTensor(),
                          transforms.Normalize(*imagenet_stats)
                          ]
)

model = learn.model.to(device)
have_texel = list()
loop = tqdm(patches)
loop.set_description("Running prediction for image")
for i, patch in enumerate(loop):
    model.eval()
    with torch.set_grad_enabled(False):
        patch = trans(patch)
        patch = torch.unsqueeze(patch, dim=0)    
        p = learn.model(patch.to('cuda'))
        if torch.argmax(p).item() == 1: # 1 indx is texel
            have_texel.append(i)
print(len(have_texel))

Running prediction for image: 100%|██████████| 1600/1600 [00:13<00:00, 115.79it/s]

0


In [ ]:
for i, patch in enumerate(patches):
    cv2.imshow("t", np.array(patches[i])) 
    btn_pressed = cv2.waitKey()  
    if btn_pressed == 113:
        cv2.destroyAllWindows()  
        break
    cv2.waitKey(0)  
    if i in have_texel:
        cv2.imshow("t", np.array(patches[i])) 
        btn_pressed = cv2.waitKey()  
        
        cv2.waitKey(0)  
        
